In [ ]:
import numpy as np
import pandas as pd
import util
import datasets2
import GradualDomainAdaptation as G

In [ ]:
# settings
steps = np.arange(3, 24, 3)
rep = 10

In [ ]:
gdamf_acc = np.full(shape=(len(steps), rep), fill_value=np.nan)
gst_acc = np.full_like(gdamf_acc, fill_value=np.nan)
wd = np.full(shape=(len(steps), steps[-1]-1), fill_value=np.nan)

for i, s in enumerate(steps):
    x_all, y_all = datasets2.make_gradual_data(steps=s)
    x_eval, y_eval = x_all.pop(), y_all.pop()
    num_init_labels, cost, budgets = util.query_settings(x_all, s-2)
    wd[i, :len(cost)] = util.mpcWD(x_all, y_all)
    for j in range(rep):
        # GDAMF without budget(=1)
        models, num_query = G.GDAMF(x_all, y_all, num_init_labels, 1, cost)
        gdamf_acc[i, j] = G.calc_accuracy(models[-1], x_eval, y_eval)
        pd.to_pickle({'wd': wd, 'acc': gdamf_acc}, 'two-moon_gdamf.pkl')
        # GST
        models, _ = G.GradualSelfTrain(x_all, y_all)
        gst_acc[i, j] = G.calc_accuracy(models[-1], x_eval, y_eval)
        pd.to_pickle({'wd': wd, 'acc': gst_acc}, 'two-moon_gst.pkl')